In [5]:

def crear_matriz(n):
    matriz = []
    for i in range(n):
        fila = []
        for j in range(n):
            fila.append(0)
        matriz.append(fila)
    return matriz

def agregar_riel(matriz, x, y, direccion):
    if direccion == 1:
        matriz[x][y] = 1 # Conecta los lados de arriba y abajo del cuadrado
    elif direccion == 2:
        matriz[x][y] = 2 # Conecta los lados de izq y der del cuadrado
    elif direccion == 3:
        matriz[x][y] = 3 # Conecta los lados de abajo y der del cuadrado
    elif direccion == 4:
        matriz[x][y] = 4 # Conecta los lados de arriba y der del cuadrado
    elif direccion == 5:
        matriz[x][y] = 5 # Conecta los lados de izq y abajo del cuadrado
    elif direccion == 6:
        matriz[x][y] = 6 # Conecta los lados de izq y arriba del cuadrado
    return matriz

def print_matriz(matriz):
    for fila in matriz:
        print(fila)

def contar_fila(matriz):
    cuenta_fila = []
    for fila in matriz:
        contador = 0
        for valor in fila:
            if valor != 0:
                contador += 1
        cuenta_fila.append(contador)
    return cuenta_fila

def contar_columnas(matriz):
    cuenta_columnas = []
    for i in range(len(matriz)):
        contador = 0
        for fila in matriz:
            if fila[i] != 0:
                contador += 1
        cuenta_columnas.append(contador)
    return cuenta_columnas


# Example usage
matriz = crear_matriz(8)
matriz = agregar_riel(matriz, 4, 0, 2)
matriz = agregar_riel(matriz, 2, 1, 4)
matriz = agregar_riel(matriz, 1, 5, 5)
matriz = agregar_riel(matriz, 4, 5, 6)
matriz = agregar_riel(matriz, 7, 4, 1)

cuenta_fila = contar_fila(matriz)
print("Cantidad en fila:", cuenta_fila, '\n')
cuenta_columnas = contar_columnas(matriz)
print("Cantidad en columna:", cuenta_columnas, '\n')
print_matriz(matriz)


Cantidad en fila: [0, 1, 1, 0, 2, 0, 0, 1] 

Cantidad en columna: [1, 1, 0, 0, 1, 2, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 5, 0, 0]
[0, 4, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 6, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]


In [3]:

import gurobipy as gp

def optimizar_matriz(matriz, limite_fila, limite_col, inicio, fin):
    n = len(matriz) #Longitud de las filas
    m = gp.Model() #Inicializa el modelo
    
    # Variable de decision que toma la fila i la columna j
    x = {}
    for i in range(n):
        for j in range(n):
            for k in range(6):
                x[i, j, k] = m.addVar(vtype=GRB.BINARY, name="x_{0}_{1}_{2}".format(i, j, k))

     # Restricción 1: La suma total de espacios en la fila y columna debe ser igual al valor de la fila y columna
    for i in range(n):
        m.addConstr(quicksum(x[i, j, k] for j in range(n) for k in range(6)) == limite_fila[i], "fila{0}".format(i))
        
    for j in range(n):
        m.addConstr(quicksum(x[i, j, k] for i in range(n) for k in range(6)) == limite_col[j], "col_{0}".format(j))

    # Restricción 2: El camino empieza en inicio y termina en fin
    m.addConstr(x[inicio[0], inicio[1], inicio[2]] == 1, name="inicio")
    m.addConstr(x[fin[0], fin[1], fin[2]] == 1, name="fin")
    
    #Almacenar rieles visitados
    rieles_visitados = set()
    
    # Restricción 3: Camino que puede tomar el riel
    for i in range(n):
        for j in range(n):
            #Caso especifico para el inicio
            if x[i,j,k] == x[inicio[0], inicio[1], inicio[2]]:
                #Cuando se ubica en la parte superior central
                if i == 0 and j != 0 and j != n-1:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la parte superior central se pueden conectar 1, 4 y 6
                        m.addConstr(x[i+1,j,1] + x[i+1,j,4] + x[i+1,j,6] == 1)
                    elif matriz[i,j] == 4:
                        #Si el riel es tipo 4 en la parte superior central se pueden conectar 2 y 5
                        m.addConstr(x[i,j+1,2] + x[i,j+1,5] == 1)
                    elif matriz[i,j] == 6:
                        #Si el riel es tipo 6 en la parte superior central se pueden conectar 2 y 3
                        m.addConstr(x[i,j-1, 2] + x[i,j-1, 3] == 1)
                        
                #Cuando se ubica en la esquina izq superior        
                elif i == 0 and j == 0:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la esquina izq superior se pueden conectar 1 y 4
                        m.addConstr(x[i+1,j,1] + x[i+1,j,4] == 1)
                    elif matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la esquina izq superior se pueden conectar 2 y 5
                        m.addConstr(x[i,j+1,2] + x[i,j+1,5] == 1)
                    elif matriz[i,j] == 4:
                        #Si el riel es tipo 4 en la esquina izq superior se pueden conectar 2 y 5
                        m.addConstr(x[i,j+1,2] + x[i,j+1,5] == 1)
                    elif matriz[i,j] == 5:
                        #Si el riel es tipo 5 en la esquina izq superior se pueden conectar 1 y 4
                        m.addConstr(x[i+1,j,1] + x[i+1,j,4] == 1)
                        
                #Cuando se ubica en la esquina der superior
                elif i == 0 and j == n-1:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la esquina der superior se pueden conectar 1 y 6
                        m.addConstr(x[i+1,j,1] + x[i+1,j,6] == 1)
                    elif matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la esquina der superior se pueden conectar 2 y 3
                        m.addConstr(x[i,j-1,2] + x[i,j-1,3] == 1)
                    elif matriz[i,j] == 3:
                        #Si el riel es tipo 3 en la esquina der superior se pueden conectar 1 y 6
                        m.addConstr(x[i+1,j,1] + x[i+1,j,6] == 1)
                    elif matriz[i,j] == 6:
                        #Si el riel es tipo 6 en la esquina der superior se pueden conectar 2 y 3
                        m.addConstr(x[i,j-1,2] + x[i,j-1,3] == 1)
                                        
                #Cuando se ubica en la parte inferior central
                elif i == n-1 and j != 0 and j != n-1:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la parte inferior central se pueden conectar 1, 3 y 5
                        m.addConstr(x[i-1,j,1] + x[i-1,j,3] + x[i-1,j,5] == 1)
                    elif matriz[i,j] == 3:
                        #Si el riel es tipo 3 en la parte inferior central se pueden conectar 2 y 6
                        m.addConstr(x[i,j+1,2] + x[i,j+1,6] == 1)
                    elif matriz[i,j] == 5:
                        #Si el riel es tipo 5 en la parte inferior central se pueden conectar 2 y 4
                        m.addConstr(x[i,j-1,2] + x[i,j-1,4]  == 1)
             
                #Cuando se ubica en la esquina izq inferior
                elif i == n-1 and j == 0:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la esquina izq inferior se pueden conectar 1 y 3
                        m.addConstr(x[i-1,j,1] + x[i-1,j,3] == 1)
                    elif matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la esquina izq inferior se pueden conectar 2 y 6
                        m.addConstr(x[i,j+1,2] + x[i,j+1,6]  == 1)
                    elif matriz[i,j] == 3:
                        #Si el riel es tipo 3 en la esquina izq inferior se pueden conectar 2 y 6
                        m.addConstr(x[i,j+1,2] + x[i,j+1,6] == 1)
                    elif matriz[i,j] == 6:
                        #Si el riel es tipo 6 en la esquina izq inferior se pueden conectar 1 y 3
                        m.addConstr(x[i-1,j,1] + x[i-1,j,3] == 1)      
                        
                #Cuando se ubica en la esquina der inferior
                elif i == n-1 and j == n-1:
                    if matriz[i,j] == 1:
                        #Si el riel es tipo 1 en la esquina der inferior se pueden conectar 1 y 5
                        m.addConstr(x[i-1,j,1] + x[i-1,j,5] == 1)
                    elif matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la esquina der inferior se pueden conectar 2 y 4
                        m.addConstr(x[i,j-1,2] + x[i,j-1,4] == 1)
                    elif matriz[i,j] == 4:
                        #Si el riel es tipo 4 en la esquina der inferior se pueden conectar 1 y 5 
                        m.addConstr(x[i-1,j,1] + x[i-1,j,5] == 1)
                    elif matriz[i,j] == 5:
                        #Si el riel es tipo 5 en la esquina der inferior se pueden conectar 2 y 4
                        m.addConstr(x[i,j-1,2] + x[i,j-1,4] == 1)    
                        
                #Cuando se ubica en la parte izq central
                elif j == 0 and i != 0 and i != n-1:
                    if matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la parte izq central se pueden conectar 2,5 y 6
                        m.addConstr(x[i,j+1,2] + x[i,j+1,5] + x[i,j+1,6] == 1)
                    elif matriz[i,j] == 5:
                        #Si el riel es tipo 5 en la parte izq central se pueden conectar 1 y 4
                        m.addConstr(x[i+1,j,1] + x[i+1,j,4] == 1)
                    elif matriz[i,j] == 6:
                        #Si el riel es tipo 6 en la parte izq central se pueden conectar 1 y 3
                        m.addConstr(x[i-1,j,1] + x[i-1,j,3] == 1)
      
                #Cuando se ubica en la parte der central
                elif j == n-1 and i != 0 and i != n-1:
                    if matriz[i,j] == 2:
                        #Si el riel es tipo 2 en la parte der central se pueden conectar 2,3 y 4
                        m.addConstr(x[i,j-1,2] + x[i,j-1,3] + x[i,j-1,4] == 1)
                    elif matriz[i,j] == 3:
                        #Si el riel es tipo 3 en la parte der central se pueden conectar 1 y 6
                        m.addConstr(x[i+1,j,1] + x[i+1,j,6] == 1)
                    elif matriz[i,j] == 4:
                        #Si el riel es tipo 4 en la parte der central se pueden conectar 1 y 5
                        m.addConstr(x[i-1,j,1] + x[i-1,j,5] == 1)
                        
            #Cuando no es la inicial pero se esta en el margen             
            elif 
                
            
            #Cuando no se esta en el margen
            elif i > 0 and j > 0 and i < n-1 and j < n-1:

                #Poner condición cuando no esta en el margen para cada riel

    #Restricción 4: Un riel no puede conectar con el margen
    #Restricción 5: Tiene que ser un camino continuo
    #Restricción 6: La solución debe pasar por los rieles existentes